# GHCND Inventory Data Prep
#### Description:
This notebook is used to prep the data contained in ghcnd-inventory.txt. Use noaa-daily-retrieve-files-ftp.ipynb to download this file among other metadta files used in the NOAA Daily Weather Project. 

System of Origin Data Source: https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-inventory.txt

#### Data Prep Operations:
- define english column names
- save to .csv file (tab separated)

#### Created by:
Nate Muth <br>
nmuth87@gmail.com

#### Created on:
8/5/2018

#### Changelog:
8/5/2018 - Initial Create Date<br>

In [1]:
import pandas as pd

In [2]:
# read inventory.txt into DataFrame
inventoryDF = pd.read_fwf('ghcnd-inventory.txt', header=None, delimiter=' '
                         , widths=[12,9,10,5,5,5]
                         , names=['StationID', 'Latitude', 'Longitude', 'Element',
                                 'FirstYear', 'LastYear']
                         #, dtypes={'WMO_ID':object}
                         )

In [3]:
inventoryDF[inventoryDF['StationID']=='USW00014942'].head()

,StationID,Latitude,Longitude,Element,FirstYear,LastYear
583588,USW00014942,41.3103,-95.8992,TMAX,1948,2018
583589,USW00014942,41.3103,-95.8992,TMIN,1948,2018
583590,USW00014942,41.3103,-95.8992,PRCP,1948,2018
583591,USW00014942,41.3103,-95.8992,SNOW,1948,2018
583592,USW00014942,41.3103,-95.8992,SNWD,1948,2018


In [4]:
# Write file to local directory
inventoryDF.to_csv('ghcnd-inventory-cleansed.csv',sep='\t')

if we're pulling in this data, we'll certainly join it to ghcnd-stations-cleansed.csv 
so that we can see stations and inventory data together

In [7]:
# read ghcnd-stations-cleansed.csv into DataFrame
stationsDF = pd.read_csv('ghcnd-stations-cleansed.csv',sep='\t',index_col='StationID')

stationsDF.head()

,Latitude,Longitude,Elevation,State,StationName,GSN_Flag,HCN_CRN_Flag,WMO_ID,CountryCode
StationID,,,,,,,,,
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN,AC
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN,AC
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0,AE
AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0,AE
AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0,AE


In [8]:
# remove duplicate columns from inventoryDF
inventoryDF = inventoryDF.filter(['StationID','Element','FirstYear','LastYear'])

In [10]:
# merge these two datasets
stations_inventory_DF = pd.merge(stationsDF,inventoryDF,on='StationID',how='left').set_index(['StationID','Element'])

stations_inventory_DF.head()

Latitude  Longitude  Elevation State  \
StationID   Element                                         
ACW00011604 TMAX      17.1167   -61.7833       10.1   NaN   
            TMIN      17.1167   -61.7833       10.1   NaN   
            PRCP      17.1167   -61.7833       10.1   NaN   
            SNOW      17.1167   -61.7833       10.1   NaN   
            SNWD      17.1167   -61.7833       10.1   NaN   

                               StationName GSN_Flag HCN_CRN_Flag  WMO_ID  \
StationID   Element                                                        
ACW00011604 TMAX     ST JOHNS COOLIDGE FLD      NaN          NaN     NaN   
            TMIN     ST JOHNS COOLIDGE FLD      NaN          NaN     NaN   
            PRCP     ST JOHNS COOLIDGE FLD      NaN          NaN     NaN   
            SNOW     ST JOHNS COOLIDGE FLD      NaN          NaN     NaN   
            SNWD     ST JOHNS COOLIDGE FLD      NaN          NaN     NaN   

                    CountryCode  FirstYear  LastYear  
StationID   Element                                   
ACW00011604 TMAX             AC     1949.0    1949.0  
            TMIN             AC     1949.0    1949.0  
            PRCP             AC     1949.0    1949.0  
            SNOW             AC     1949.0    1949.0  
            SNWD             AC     1949.0    1949.0

In [11]:
# write stations_inventory_DF to a .csv file in the local directory
stations_inventory_DF.to_csv('ghcnd-stations-inventory-cleansed.csv',sep='\t')